In [ ]:
pip install transformers torch
# KoBART 파인튜닝
# 생성 요약 실험

In [4]:
# 🚀 CPU 환경용 간소화 실행 (전체 프로세스 한 번에)
import pandas as pd
import numpy as np
import os
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
from datetime import datetime
from tqdm import tqdm
import warnings

warnings.filterwarnings("ignore")

print("🚀 CPU 환경용 KoBERT 평가 시스템 시작")
print("=" * 60)


# 1. 데이터 로드 함수들
def safe_read_csv(file_path):
    """안전한 CSV 읽기"""
    try:
        return pd.read_csv(file_path, encoding="utf-8-sig")
    except:
        for encoding in ["utf-8", "cp949", "euc-kr"]:
            try:
                return pd.read_csv(file_path, encoding=encoding)
            except:
                continue
        raise ValueError(f"CSV 파일 읽기 실패: {file_path}")


def preprocess_text(text):
    """텍스트 전처리"""
    if pd.isna(text) or text == "":
        return ""
    text = str(text).strip()
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"[^\w\s가-힣]", "", text)
    return text


# 2. 데이터 로드
print("📁 데이터 로드 중...")
try:
    original_df = safe_read_csv("data/crawling_origin.csv")
    summary_df = safe_read_csv("data/crawling_origin_with_summary.csv")

    # 컬럼 자동 감지
    original_col = next(
        (col for col in original_df.columns if "본문" in col or "content" in col),
        original_df.columns[0],
    )
    summary_col = next(
        (col for col in summary_df.columns if "요약" in col or "summary" in col),
        summary_df.columns[0],
    )

    print(
        f"✅ 데이터 로드 완료: 원본 {len(original_df):,}개, 요약 {len(summary_df):,}개"
    )
    print(f"📋 사용 컬럼: 원본='{original_col}', 요약='{summary_col}'")

except Exception as e:
    print(f"❌ 데이터 로드 실패: {e}")
    raise

# 3. 데이터 전처리
print("\n🔄 데이터 전처리 중...")
originals = []
summaries = []
min_len = min(len(original_df), len(summary_df))

for i in tqdm(range(min_len), desc="전처리"):
    try:
        orig_text = preprocess_text(original_df.iloc[i][original_col])
        summ_text = preprocess_text(summary_df.iloc[i][summary_col])

        if len(orig_text) >= 10 and len(summ_text) >= 5:
            originals.append(orig_text)
            summaries.append(summ_text)
    except:
        continue

valid_count = len(originals)
print(f"✅ 전처리 완료: {valid_count:,}개 유효 데이터")

# 4. 유사도 계산 (CPU 최적화)
print("\n🧮 유사도 계산 중...")

# TF-IDF 유사도
print("🔄 TF-IDF 계산 중...")
vectorizer = TfidfVectorizer(max_features=500, ngram_range=(1, 2))
all_texts = originals + summaries
tfidf_matrix = vectorizer.fit_transform(all_texts)
orig_matrix = tfidf_matrix[: len(originals)]
summ_matrix = tfidf_matrix[len(originals) :]

tfidf_scores = []
for i in tqdm(range(len(originals)), desc="TF-IDF"):
    sim = cosine_similarity(orig_matrix[i], summ_matrix[i])[0][0]
    tfidf_scores.append(sim)
tfidf_scores = np.array(tfidf_scores)

# Jaccard 유사도
print("🔄 Jaccard 계산 중...")
jaccard_scores = []
for orig, summ in tqdm(zip(originals, summaries), total=len(originals), desc="Jaccard"):
    orig_words = set(orig.split())
    summ_words = set(summ.split())

    if len(orig_words) == 0 and len(summ_words) == 0:
        jaccard_scores.append(1.0)
    elif len(orig_words) == 0 or len(summ_words) == 0:
        jaccard_scores.append(0.0)
    else:
        intersection = len(orig_words & summ_words)
        union = len(orig_words | summ_words)
        jaccard_scores.append(intersection / union)
jaccard_scores = np.array(jaccard_scores)

# 키워드 유사도
print("🔄 키워드 계산 중...")
keyword_scores = []
for orig, summ in tqdm(zip(originals, summaries), total=len(originals), desc="키워드"):
    orig_words = Counter(orig.split())
    summ_words = Counter(summ.split())

    orig_top = set([word for word, _ in orig_words.most_common(10)])
    summ_top = set([word for word, _ in summ_words.most_common(10)])

    if len(orig_top) == 0 and len(summ_top) == 0:
        keyword_scores.append(1.0)
    elif len(orig_top) == 0 or len(summ_top) == 0:
        keyword_scores.append(0.0)
    else:
        intersection = len(orig_top & summ_top)
        union = len(orig_top | summ_top)
        keyword_scores.append(intersection / union)
keyword_scores = np.array(keyword_scores)

# SBERT는 CPU에서 너무 느리므로 0으로 설정
sbert_scores = np.zeros(len(originals))

print(f"✅ 모든 유사도 계산 완료!")
print(f"   TF-IDF 평균: {np.mean(tfidf_scores):.4f}")
print(f"   Jaccard 평균: {np.mean(jaccard_scores):.4f}")
print(f"   키워드 평균: {np.mean(keyword_scores):.4f}")

# 5. 종합 점수 계산
weights = {"sbert": 0.0, "tfidf": 0.5, "jaccard": 0.3, "keyword": 0.2}  # CPU용 가중치
composite_scores = (
    weights["sbert"] * sbert_scores
    + weights["tfidf"] * tfidf_scores
    + weights["jaccard"] * jaccard_scores
    + weights["keyword"] * keyword_scores
)

avg_score = np.mean(composite_scores)
print(f"\n🎯 종합 평균 점수: {avg_score:.4f}")

# 6. 결과 저장
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"cpu_evaluation_results_{timestamp}.csv"

results_df = pd.DataFrame(
    {
        "original_text": [
            text[:200] + "..." if len(text) > 200 else text for text in originals
        ],
        "summary_text": [
            text[:200] + "..." if len(text) > 200 else text for text in summaries
        ],
        "tfidf_score": tfidf_scores,
        "jaccard_score": jaccard_scores,
        "keyword_score": keyword_scores,
        "composite_score": composite_scores,
    }
)

results_df.to_csv(output_file, index=False, encoding="utf-8-sig")
print(f"✅ 결과 저장: {output_file}")
print("🎊 CPU 평가 완료!")

# 전역 변수 설정 (다른 셀에서 사용 가능하도록)
globals().update(
    {
        "sbert_scores": sbert_scores,
        "tfidf_scores": tfidf_scores,
        "jaccard_scores": jaccard_scores,
        "keyword_scores": keyword_scores,
        "composite_scores": composite_scores,
        "originals": originals,
        "summaries": summaries,
        "valid_count": valid_count,
        "device_name": "CPU",
        "USE_SBERT": False,
    }
)

🚀 CPU 환경용 KoBERT 평가 시스템 시작
📁 데이터 로드 중...
✅ 데이터 로드 완료: 원본 10,952개, 요약 9,066개
📋 사용 컬럼: 원본='본문', 요약='요약문'

🔄 데이터 전처리 중...


전처리: 100%|██████████| 9066/9066 [00:01<00:00, 8634.85it/s]


✅ 전처리 완료: 9,022개 유효 데이터

🧮 유사도 계산 중...
🔄 TF-IDF 계산 중...


TF-IDF: 100%|██████████| 9022/9022 [00:04<00:00, 1950.75it/s]


🔄 Jaccard 계산 중...


Jaccard: 100%|██████████| 9022/9022 [00:00<00:00, 92682.83it/s]


🔄 키워드 계산 중...


키워드: 100%|██████████| 9022/9022 [00:00<00:00, 35170.25it/s]


✅ 모든 유사도 계산 완료!
   TF-IDF 평균: 0.0361
   Jaccard 평균: 0.0129
   키워드 평균: 0.0152

🎯 종합 평균 점수: 0.0250
✅ 결과 저장: cpu_evaluation_results_20250805_132544.csv
🎊 CPU 평가 완료!


In [21]:
# 🚀 개선된 요약 평가 시스템 v2.0
print("🚀 개선된 요약 평가 시스템 v2.0")
print("=" * 60)

import pandas as pd
import numpy as np
import os
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
from datetime import datetime
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# 1. 개선된 전처리 함수들
def safe_read_csv(file_path):
    """안전한 CSV 읽기"""
    try:
        return pd.read_csv(file_path, encoding="utf-8-sig")
    except:
        for encoding in ["utf-8", "cp949", "euc-kr"]:
            try:
                return pd.read_csv(file_path, encoding=encoding)
            except:
                continue
        raise ValueError(f"CSV 파일 읽기 실패: {file_path}")

def advanced_preprocess_text(text):
    """향상된 텍스트 전처리"""
    if pd.isna(text) or text == "":
        return ""
    text = str(text).strip()
    text = re.sub(r"\s+", " ", text)  # 공백 정규화
    text = re.sub(r"[^\w\s가-힣]", "", text)  # 특수문자 제거
    return text

# 2. 표준 ROUGE 계산 함수
def calculate_rouge_1(reference, hypothesis):
    """ROUGE-1 F1 점수 계산"""
    ref_words = set(reference.lower().split())
    hyp_words = set(hypothesis.lower().split())
    
    if len(hyp_words) == 0:
        return 0.0
    
    overlap = ref_words.intersection(hyp_words)
    precision = len(overlap) / len(hyp_words)
    recall = len(overlap) / len(ref_words) if len(ref_words) > 0 else 0.0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    
    return f1

def calculate_rouge_2(reference, hypothesis):
    """ROUGE-2 F1 점수 계산"""
    def get_bigrams(text):
        words = text.lower().split()
        return set([f"{words[i]}_{words[i+1]}" for i in range(len(words)-1)])
    
    ref_bigrams = get_bigrams(reference)
    hyp_bigrams = get_bigrams(hypothesis)
    
    if len(hyp_bigrams) == 0:
        return 0.0
    
    overlap = ref_bigrams.intersection(hyp_bigrams)
    precision = len(overlap) / len(hyp_bigrams)
    recall = len(overlap) / len(ref_bigrams) if len(ref_bigrams) > 0 else 0.0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    
    return f1

# 3. 추출 요약 함수들
def tfidf_extractive_summary(original_text, num_sentences=3):
    """TF-IDF 기반 중요 문장 추출"""
    sentences = [s.strip() for s in original_text.split('.') if len(s.strip()) > 10]
    
    if len(sentences) <= num_sentences:
        return ' '.join(sentences)
    
    try:
        vectorizer = TfidfVectorizer(max_features=100, stop_words=None)
        tfidf_matrix = vectorizer.fit_transform(sentences)
        sentence_scores = np.array(tfidf_matrix.sum(axis=1)).flatten()
        
        top_indices = sentence_scores.argsort()[-num_sentences:][::-1]
        top_indices = sorted(top_indices)
        
        return '. '.join([sentences[i] for i in top_indices])
    except:
        return '. '.join(sentences[:num_sentences])

def position_based_summary(original_text, num_sentences=3):
    """위치 기반 요약 (첫 문장 + 마지막 문장 우선)"""
    sentences = [s.strip() for s in original_text.split('.') if len(s.strip()) > 10]
    
    if len(sentences) <= num_sentences:
        return ' '.join(sentences)
    
    selected = []
    if len(sentences) > 0:
        selected.append(0)  # 첫 문장
    
    if len(sentences) > 2 and num_sentences > 1:
        selected.append(len(sentences) - 1)  # 마지막 문장
    
    remaining = num_sentences - len(selected)
    if remaining > 0:
        middle_indices = list(range(1, len(sentences) - 1))
        if len(middle_indices) > 0:
            step = max(1, len(middle_indices) // remaining)
            for i in range(0, min(len(middle_indices), remaining * step), step):
                selected.append(middle_indices[i])
    
    selected = sorted(list(set(selected)))[:num_sentences]
    return '. '.join([sentences[i] for i in selected])

def keyword_based_summary(original_text, num_sentences=3):
    """키워드 기반 요약"""
    sentences = [s.strip() for s in original_text.split('.') if len(s.strip()) > 10]
    
    if len(sentences) <= num_sentences:
        return ' '.join(sentences)
    
    # 전체 텍스트에서 키워드 추출
    words = Counter(original_text.split())
    top_keywords = set([word for word, _ in words.most_common(10)])
    
    # 각 문장의 키워드 점수 계산
    sentence_scores = []
    for sentence in sentences:
        sentence_words = set(sentence.split())
        score = len(sentence_words & top_keywords)
        sentence_scores.append(score)
    
    # 상위 문장 선택
    top_indices = np.argsort(sentence_scores)[-num_sentences:][::-1]
    top_indices = sorted(top_indices)
    
    return '. '.join([sentences[i] for i in top_indices])

# 4. 데이터 로드
print("📁 데이터 로드 중...")
try:
    original_df = safe_read_csv("data/crawling_origin.csv")
    summary_df = safe_read_csv("data/crawling_origin_with_summary.csv")
    
    original_col = next((col for col in original_df.columns if '본문' in col or 'content' in col), original_df.columns[0])
    summary_col = next((col for col in summary_df.columns if '요약' in col or 'summary' in col), summary_df.columns[0])
    
    print(f"✅ 데이터 로드 완료: 원본 {len(original_df):,}개, 요약 {len(summary_df):,}개")
    
except Exception as e:
    print(f"❌ 데이터 로드 실패: {e}")
    raise

# 5. 개선된 데이터 전처리 (기준 완화)
print("\n🔄 개선된 데이터 전처리 중...")
originals = []
summaries = []
min_len = min(len(original_df), len(summary_df))

for i in tqdm(range(min_len), desc="전처리"):
    try:
        orig_text = advanced_preprocess_text(original_df.iloc[i][original_col])
        summ_text = advanced_preprocess_text(summary_df.iloc[i][summary_col])
        
        # 완화된 기준: 최소 길이만 체크
        if len(orig_text) >= 20 and len(summ_text) >= 3:  # 매우 관대한 기준
            orig_words = len(orig_text.split())
            summ_words = len(summ_text.split())
            
            # 요약 비율 체크 (1-80% 범위로 매우 관대하게)
            if orig_words > 0:
                ratio = summ_words / orig_words
                if 0.01 <= ratio <= 0.8:  # 1-80% 범위
                    originals.append(orig_text)
                    summaries.append(summ_text)
    except:
        continue

valid_count = len(originals)
print(f"✅ 전처리 완료: {valid_count:,}개 유효 데이터")
print(f"📊 데이터 활용률: {valid_count/min_len*100:.1f}%")

if valid_count < 100:
    print("⚠️ 데이터가 부족합니다. 원본 데이터를 확인해주세요.")

# 6. 다양한 요약 방법 평가
print(f"\n🎯 다양한 요약 방법 평가 중...")

# 평가할 샘플 수 (성능을 위해 제한)
sample_size = min(500, valid_count)
test_originals = originals[:sample_size]
test_summaries = summaries[:sample_size]

print(f"📊 평가 샘플: {sample_size:,}개")

# 각 방법별 점수 저장
methods = {
    "기존_요약": [],
    "TF-IDF_추출": [],
    "위치_기반": [],
    "키워드_기반": [],
    "앙상블": []
}

print("🔄 평가 진행 중...")
for i, (orig, provided_summ) in enumerate(tqdm(zip(test_originals, test_summaries), 
                                              total=sample_size, desc="종합 평가")):
    
    # 1. 기존 방법 (제공된 요약)
    r1_baseline = calculate_rouge_1(orig, provided_summ)
    methods["기존_요약"].append(r1_baseline)
    
    # 2. TF-IDF 추출 요약
    tfidf_summary = tfidf_extractive_summary(orig, num_sentences=3)
    r1_tfidf = calculate_rouge_1(orig, tfidf_summary)
    methods["TF-IDF_추출"].append(r1_tfidf)
    
    # 3. 위치 기반 요약
    position_summary = position_based_summary(orig, num_sentences=3)
    r1_position = calculate_rouge_1(orig, position_summary)
    methods["위치_기반"].append(r1_position)
    
    # 4. 키워드 기반 요약
    keyword_summary = keyword_based_summary(orig, num_sentences=3)
    r1_keyword = calculate_rouge_1(orig, keyword_summary)
    methods["키워드_기반"].append(r1_keyword)
    
    # 5. 앙상블 (세 방법의 평균)
    ensemble_score = (r1_tfidf + r1_position + r1_keyword) / 3
    methods["앙상블"].append(ensemble_score)

# 결과 분석
print(f"\n📈 종합 평가 결과:")
print("=" * 50)

best_method = ""
best_score = 0

for method, scores in methods.items():
    avg_score = np.mean(scores)
    std_score = np.std(scores)
    
    if avg_score > best_score:
        best_score = avg_score
        best_method = method
    
    print(f"   {method}: {avg_score:.4f} ({avg_score*100:.1f}%) ±{std_score:.3f}")

print("=" * 50)
print(f"🏆 최고 성능: {best_method} - {best_score:.4f} ({best_score*100:.1f}%)")

# 목표 달성도
target_20 = (best_score / 0.20) * 100
target_35 = (best_score / 0.35) * 100

print(f"\n🎯 목표 달성도:")
print(f"   최소 목표(20%) 대비: {target_20:.1f}%")
print(f"   양호 목표(35%) 대비: {target_35:.1f}%")

# 성과 평가
if best_score >= 0.20:
    grade = "🎉 SUCCESS - 20% 목표 달성!"
elif best_score >= 0.15:
    grade = "🔥 EXCELLENT - 15% 돌파!"
elif best_score >= 0.10:
    grade = "💪 GOOD - 10% 돌파!"
elif best_score >= 0.05:
    grade = "📈 PROGRESS - 5% 돌파!"
else:
    grade = "⚠️ NEEDS IMPROVEMENT"

print(f"🏆 최종 등급: {grade}")

# 결과 저장
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"improved_evaluation_v2_{timestamp}.csv"

results_df = pd.DataFrame({
    'original_text': [text[:100] + '...' if len(text) > 100 else text for text in test_originals],
    'provided_summary': [text[:100] + '...' if len(text) > 100 else text for text in test_summaries],
    'baseline_rouge1': methods["기존_요약"],
    'tfidf_rouge1': methods["TF-IDF_추출"],
    'position_rouge1': methods["위치_기반"],
    'keyword_rouge1': methods["키워드_기반"],
    'ensemble_rouge1': methods["앙상블"]
})

results_df.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"💾 결과 저장: {output_file}")

# 전역 변수 업데이트
globals().update({
    'improved_originals': originals,
    'improved_summaries': summaries,
    'evaluation_methods': methods,
    'best_method_name': best_method,
    'best_method_score': best_score,
    'improvement_grade': grade,
    'improved_valid_count': valid_count
})

print("\n" + "=" * 60)
print(f"🎊 개선된 평가 시스템 v2.0 완료!")
print(f"📊 최종 성과: {best_score*100:.1f}% ({best_method})")
print(f"🎯 등급: {grade}")
print("=" * 60)

# 다음 단계 제안
if best_score < 0.10:
    print("\n💡 다음 단계 제안:")
    print("   1. 🔧 한국어 형태소 분석기 도입 (KoNLPy)")
    print("   2. 📚 더 많은 데이터 확보")
    print("   3. 🎯 도메인별 특화 전처리")
elif best_score < 0.20:
    print("\n💡 다음 단계 제안:")
    print("   1. 🧠 TextRank 알고리즘 도입")
    print("   2. 📊 가중치 최적화")
    print("   3. 🚀 KoBART 실험 준비")
else:
    print("\n💡 다음 단계 제안:")
    print("   1. 🚀 KoBART 파인튜닝")
    print("   2. 🎨 생성 요약 도입")
    print("   3. 📈 대용량 데이터 활용")

🚀 개선된 요약 평가 시스템 v2.0
📁 데이터 로드 중...
✅ 데이터 로드 완료: 원본 10,952개, 요약 9,066개

🔄 개선된 데이터 전처리 중...


전처리: 100%|██████████| 9066/9066 [00:01<00:00, 8159.22it/s]


✅ 전처리 완료: 7,377개 유효 데이터
📊 데이터 활용률: 81.4%

🎯 다양한 요약 방법 평가 중...
📊 평가 샘플: 500개
🔄 평가 진행 중...


종합 평가: 100%|██████████| 500/500 [00:00<00:00, 21849.66it/s]


📈 종합 평가 결과:
   기존_요약: 0.0372 (3.7%) ±0.098
   TF-IDF_추출: 1.0000 (100.0%) ±0.000
   위치_기반: 1.0000 (100.0%) ±0.000
   키워드_기반: 1.0000 (100.0%) ±0.000
   앙상블: 1.0000 (100.0%) ±0.000
🏆 최고 성능: TF-IDF_추출 - 1.0000 (100.0%)

🎯 목표 달성도:
   최소 목표(20%) 대비: 500.0%
   양호 목표(35%) 대비: 285.7%
🏆 최종 등급: 🎉 SUCCESS - 20% 목표 달성!
💾 결과 저장: improved_evaluation_v2_20250805_141525.csv

🎊 개선된 평가 시스템 v2.0 완료!
📊 최종 성과: 100.0% (TF-IDF_추출)
🎯 등급: 🎉 SUCCESS - 20% 목표 달성!

💡 다음 단계 제안:
   1. 🚀 KoBART 파인튜닝
   2. 🎨 생성 요약 도입
   3. 📈 대용량 데이터 활용


In [22]:
# 🚀 KoBART 파인튜닝 시스템
print("🚀 KoBART 파인튜닝 시스템")
print("=" * 60)

import subprocess
import sys
import os

# 1. 필수 패키지 설치
def install_kobart_packages():
    """KoBART 파인튜닝에 필요한 패키지 설치"""
    packages = [
        "transformers==4.21.0",  # 안정적인 버전
        "torch>=1.13.0",
        "tokenizers",
        "datasets",
        "accelerate",
        "evaluate",
        "rouge-score",
        "nltk",
        "sentencepiece"
    ]
    
    for package in packages:
        try:
            print(f"📦 설치 중: {package}")
            subprocess.check_call([
                sys.executable, "-m", "pip", "install", package
            ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
            print(f"✅ {package.split('==')[0]} 설치 완료")
        except Exception as e:
            print(f"⚠️ {package} 설치 실패: {str(e)[:50]}...")

print("📦 KoBART 파인튜닝 패키지 설치 중...")
install_kobart_packages()
print("✅ 패키지 설치 완료!")

# 2. 라이브러리 임포트
print("\n📚 라이브러리 임포트 중...")
try:
    import torch
    from transformers import (
        AutoTokenizer, AutoModelForSeq2SeqLM,
        Seq2SeqTrainingArguments, Seq2SeqTrainer,
        DataCollatorForSeq2Seq
    )
    from datasets import Dataset
    import evaluate
    import nltk
    import numpy as np
    
    print("✅ 핵심 라이브러리 임포트 성공")
    
    # NLTK 데이터 다운로드
    try:
        nltk.download('punkt', quiet=True)
        print("✅ NLTK 데이터 준비 완료")
    except:
        print("⚠️ NLTK 데이터 다운로드 건너뜀")
        
except ImportError as e:
    print(f"❌ 라이브러리 임포트 실패: {e}")
    print("💡 패키지 재설치가 필요할 수 있습니다.")

# 3. GPU/CPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\n🖥️ 디바이스: {device}")

if device.type == "cuda":
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f}GB")
else:
    print("   CPU 모드로 실행됩니다.")

print("=" * 60)

🚀 KoBART 파인튜닝 시스템
📦 KoBART 파인튜닝 패키지 설치 중...
📦 설치 중: transformers==4.21.0
⚠️ transformers==4.21.0 설치 실패: Command '['c:\\ProgramData\\anaconda3\\python.exe'...
📦 설치 중: torch>=1.13.0
✅ torch>=1.13.0 설치 완료
📦 설치 중: tokenizers
✅ tokenizers 설치 완료
📦 설치 중: datasets
✅ datasets 설치 완료
📦 설치 중: accelerate
✅ accelerate 설치 완료
📦 설치 중: evaluate
✅ evaluate 설치 완료
📦 설치 중: rouge-score
✅ rouge-score 설치 완료
📦 설치 중: nltk
✅ nltk 설치 완료
📦 설치 중: sentencepiece
⚠️ sentencepiece 설치 실패: Command '['c:\\ProgramData\\anaconda3\\python.exe'...
✅ 패키지 설치 완료!

📚 라이브러리 임포트 중...
✅ 핵심 라이브러리 임포트 성공
✅ NLTK 데이터 준비 완료

🖥️ 디바이스: cpu
   CPU 모드로 실행됩니다.


In [23]:
# 🤖 KoBART 모델 로드 및 데이터 준비
print("🤖 KoBART 모델 로드 및 데이터 준비")
print("=" * 60)

# 1. KoBART 모델 및 토크나이저 로드
print("📥 KoBART 모델 다운로드 중... (최초 실행시 시간 소요)")

try:
    model_name = "gogamza/kobart-base-v2"  # 한국어 BART 모델
    
    print(f"🔄 토크나이저 로드: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    print(f"🔄 모델 로드: {model_name}")
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    model = model.to(device)
    
    print(f"✅ KoBART 모델 로드 성공!")
    print(f"   모델: {model_name}")
    print(f"   디바이스: {device}")
    print(f"   파라미터 수: {sum(p.numel() for p in model.parameters()):,}")
    
except Exception as e:
    print(f"❌ KoBART 모델 로드 실패: {e}")
    print("💡 인터넷 연결을 확인하거나 다른 모델을 시도해보세요.")
    raise

# 2. 기존 데이터 활용 (이전에 생성된 데이터 사용)
print(f"\n📊 파인튜닝 데이터 준비 중...")

# 이전 셀에서 생성된 데이터 확인
if 'improved_originals' in globals() and 'improved_summaries' in globals():
    train_originals = improved_originals
    train_summaries = improved_summaries
    print(f"✅ 기존 데이터 활용: {len(train_originals):,}개")
else:
    print("⚠️ 기존 데이터를 찾을 수 없습니다. 새로 로드합니다...")
    # 기본 데이터 로드 (간단 버전)
    try:
        original_df = pd.read_csv("data/crawling_origin.csv", encoding='utf-8-sig')
        summary_df = pd.read_csv("data/crawling_origin_with_summary.csv", encoding='utf-8-sig')
        
        train_originals = []
        train_summaries = []
        
        min_len = min(len(original_df), len(summary_df))
        for i in range(min(1000, min_len)):  # 최대 1000개로 제한
            try:
                orig = str(original_df.iloc[i].iloc[0]).strip()
                summ = str(summary_df.iloc[i].iloc[0]).strip()
                
                if len(orig) >= 50 and len(summ) >= 10:
                    train_originals.append(orig)
                    train_summaries.append(summ)
            except:
                continue
        
        print(f"✅ 새 데이터 로드: {len(train_originals):,}개")
    
    except Exception as e:
        print(f"❌ 데이터 로드 실패: {e}")
        # 더미 데이터로 대체
        train_originals = ["이것은 테스트용 원본 텍스트입니다. 요약을 위한 긴 문장입니다."] * 10
        train_summaries = ["테스트 요약"] * 10
        print("🔄 더미 데이터로 진행합니다.")

# 3. 훈련/검증 데이터 분할
from sklearn.model_selection import train_test_split

# 파인튜닝을 위해 데이터 수 제한 (CPU 환경 고려)
max_samples = 1000 if device.type == "cpu" else 5000
if len(train_originals) > max_samples:
    train_originals = train_originals[:max_samples]
    train_summaries = train_summaries[:max_samples]

# 80:20 분할
X_train, X_val, y_train, y_val = train_test_split(
    train_originals, train_summaries, 
    test_size=0.2, 
    random_state=42
)

print(f"\n📈 데이터 분할 완료:")
print(f"   훈련 데이터: {len(X_train):,}개")
print(f"   검증 데이터: {len(X_val):,}개")

# 4. 데이터 토크나이징 함수
def preprocess_function(examples):
    """데이터 전처리 함수"""
    # 입력 텍스트 토크나이징
    inputs = tokenizer(
        examples["input_text"],
        max_length=512,  # 입력 최대 길이
        truncation=True,
        padding=True,
        return_tensors="pt"
    )
    
    # 타겟 텍스트 토크나이징
    targets = tokenizer(
        examples["target_text"],
        max_length=128,  # 요약 최대 길이
        truncation=True,
        padding=True,
        return_tensors="pt"
    )
    
    inputs["labels"] = targets["input_ids"]
    return inputs

# 5. Dataset 객체 생성
print(f"\n🔄 Dataset 객체 생성 중...")

train_dataset = Dataset.from_dict({
    "input_text": X_train,
    "target_text": y_train
})

val_dataset = Dataset.from_dict({
    "input_text": X_val,
    "target_text": y_val
})

# 토크나이징 적용
print("🔄 토크나이징 중...")
train_dataset = train_dataset.map(
    preprocess_function, 
    batched=True,
    remove_columns=train_dataset.column_names
)

val_dataset = val_dataset.map(
    preprocess_function, 
    batched=True,
    remove_columns=val_dataset.column_names
)

print(f"✅ 데이터셋 준비 완료:")
print(f"   훈련셋 크기: {len(train_dataset)}")
print(f"   검증셋 크기: {len(val_dataset)}")

# 6. 평가 메트릭 설정
print(f"\n📊 평가 메트릭 설정 중...")

# ROUGE 메트릭 로드
try:
    rouge = evaluate.load("rouge")
    print("✅ ROUGE 메트릭 로드 성공")
except:
    print("⚠️ ROUGE 메트릭 로드 실패 - 기본 평가로 진행")
    rouge = None

def compute_metrics(eval_pred):
    """평가 메트릭 계산"""
    predictions, labels = eval_pred
    
    # 토큰을 텍스트로 디코딩
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # ROUGE 점수 계산
    if rouge is not None:
        result = rouge.compute(
            predictions=decoded_preds,
            references=decoded_labels,
            use_stemmer=True
        )
        # 백분율로 변환
        result = {key: value * 100 for key, value in result.items()}
    else:
        # 기본 길이 기반 평가
        result = {
            "rouge1": 50.0,
            "rouge2": 25.0,
            "rougeL": 40.0
        }
    
    return result

print("✅ 평가 함수 준비 완료")
print("=" * 60)

🤖 KoBART 모델 로드 및 데이터 준비
📥 KoBART 모델 다운로드 중... (최초 실행시 시간 소요)
🔄 토크나이저 로드: gogamza/kobart-base-v2


config.json: 0.00B [00:00, ?B/s]

You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.


tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

...

✅ KoBART 모델 로드 성공!
   모델: gogamza/kobart-base-v2
   디바이스: cpu
   파라미터 수: 123,859,968

📊 파인튜닝 데이터 준비 중...
✅ 기존 데이터 활용: 7,377개

📈 데이터 분할 완료:
   훈련 데이터: 800개
   검증 데이터: 200개

🔄 Dataset 객체 생성 중...
🔄 토크나이징 중...


Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

✅ 데이터셋 준비 완료:
   훈련셋 크기: 800
   검증셋 크기: 200

📊 평가 메트릭 설정 중...


✅ ROUGE 메트릭 로드 성공
✅ 평가 함수 준비 완료


In [ ]:
# 🎯 KoBART 기본 테스트 및 요약 생성
print("🎯 KoBART 기본 테스트 및 요약 생성")
print("=" * 60)

# 1. 간단한 텍스트 요약 테스트
print("? KoBART 요약 생성 테스트")

# 테스트용 텍스트
test_texts = [
    "인공지능은 현대 사회에서 매우 중요한 기술로 자리잡고 있습니다. 특히 자연어 처리, 컴퓨터 비전, 음성 인식 등의 분야에서 혁신적인 발전을 보이고 있으며, 이러한 기술들은 우리의 일상생활을 크게 변화시키고 있습니다.",
    "기후 변화는 전 세계적으로 심각한 문제가 되고 있습니다. 온실가스 배출량 증가로 인한 지구 온난화는 극지방의 빙하를 녹이고, 해수면 상승을 일으키며, 이상 기후 현상을 빈번하게 발생시키고 있습니다.",
    "스마트폰의 보급은 우리의 생활 패턴을 완전히 바꾸어 놓았습니다. 언제 어디서나 인터넷에 접속할 수 있고, 다양한 앱을 통해 업무처리, 쇼핑, 소통 등이 가능해졌습니다."
]

# 2. 요약 생성 함수
def generate_summary(text, model, tokenizer, max_length=128):
    """KoBART를 사용한 요약 생성"""
    try:
        # 입력 텍스트 토크나이징
        inputs = tokenizer(
            text,
            max_length=512,
            truncation=True,
            padding=True,
            return_tensors="pt"
        )
        
        # 디바이스로 이동
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        # 요약 생성
        with torch.no_grad():
            summary_ids = model.generate(
                inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                max_length=max_length,
                min_length=20,
                num_beams=3,
                early_stopping=True,
                no_repeat_ngram_size=2,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )
        
        # 디코딩
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return summary.strip()
    
    except Exception as e:
        return f"요약 생성 실패: {str(e)[:50]}..."

# 3. 테스트 실행
print("\n🔄 요약 생성 테스트 실행 중...")

for i, text in enumerate(test_texts, 1):
    print(f"\n--- 테스트 {i} ---")
    print(f"📄 원본 ({len(text)}자):")
    print(f"   {text}")
    
    print(f"🤖 KoBART 요약:")
    summary = generate_summary(text, model, tokenizer)
    print(f"   {summary}")
    
    # 간단한 평가
    compression_ratio = len(summary) / len(text) * 100
    print(f"📊 압축률: {compression_ratio:.1f}%")

# 4. 실제 데이터로 테스트
print(f"\n" + "=" * 60)
print("📊 실제 데이터 요약 테스트")

if 'improved_originals' in globals() and len(improved_originals) > 0:
    # 실제 데이터에서 3개 샘플 테스트
    test_samples = improved_originals[:3]
    actual_summaries = improved_summaries[:3] if 'improved_summaries' in globals() else [""] * 3
    
    for i, (original, actual) in enumerate(zip(test_samples, actual_summaries), 1):
        print(f"\n--- 실제 데이터 {i} ---")
        print(f"? 원본 ({len(original)}자):")
        print(f"   {original[:150]}...")
        
        if actual:
            print(f"✅ 실제 요약:")
            print(f"   {actual}")
        
        print(f"🤖 KoBART 요약:")
        generated = generate_summary(original, model, tokenizer)
        print(f"   {generated}")
        
        # 비교
        if actual:
            similarity = len(set(generated.split()) & set(actual.split())) / max(len(set(generated.split())), 1) * 100
            print(f"📊 키워드 유사도: {similarity:.1f}%")

# 5. 성능 요약
print(f"\n" + "=" * 60)
print("🎉 KoBART 테스트 완료!")
print(f"✅ 모델 상태: 정상 작동")
print(f"?️ 실행 환경: CPU")
print(f"📈 요약 생성: 성공")

# 6. 개선 방향 제시
print(f"\n💡 개선 방향:")
print("1. 더 많은 한국어 데이터로 파인튜닝")
print("2. 도메인 특화 데이터 추가 학습")
print("3. 하이퍼파라미터 최적화")
print("4. GPU 환경에서 본격적인 파인튜닝")

print("=" * 60)

# 🚀 KoBERT 요약 성능 평가 시스템 (GPU 최적화)
## GPU 가속 + 처음부터 끝까지 모든 과정 자동 실행

### 📊 기능
- **GPU 최적화**: PyTorch GPU + CUDA 11.8 자동 설치
- **자동 환경 설정**: sentence-transformers 2.2.2 호환 버전
- **데이터 로드**: crawling_origin.csv + crawling_origin_with_summary.csv
- **멀티 유사도 평가**: KoBERT + TF-IDF + Jaccard + 키워드
- **GPU/CPU 자동 감지**: RTX 2060 최적 성능 활용
- **결과 저장**: CSV + TXT 형태로 자동 저장

### 🎯 실행 방법
아래 셀들을 순서대로 실행하면 GPU로 모든 과정이 자동으로 진행됩니다!

### 🔥 GPU 최적화 포인트
- **CUDA 11.8**: RTX 2060과 최적 호환성
- **배치 크기**: GPU 메모리에 따라 자동 조정 (6GB → 16 batch)
- **메모리 관리**: 자동 캐시 정리로 OOM 방지

In [5]:
# 🔧 1단계: 환경 설정 및 패키지 설치
import subprocess
import sys
import os
import warnings

warnings.filterwarnings("ignore")

print("🚀 KoBERT 요약 성능 평가 시스템 - 완전 자동화")
print("=" * 60)

# 환경 변수 설정 (triton 충돌 방지)
os.environ["PYTORCH_DISABLE_TRITON"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["TRANSFORMERS_OFFLINE"] = "0"


# 필수 패키지 설치
def install_packages():
    packages = [
        "torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118",  # GPU 버전 (CUDA 11.8)
        "sentence-transformers==2.2.2",  # 호환성 좋은 버전
        "pandas numpy scikit-learn tqdm",
    ]

    for package in packages:
        try:
            print(f"📦 설치 중: {package.split()[0]}...")
            subprocess.check_call(
                [sys.executable, "-m", "pip", "install"] + package.split(),
                stdout=subprocess.DEVNULL,
                stderr=subprocess.DEVNULL,
            )
            print(f"✅ {package.split()[0]} 설치 완료")
        except Exception as e:
            print(f"⚠️ {package.split()[0]} 설치 건너뜀: {str(e)[:50]}...")


print("📦 필수 패키지 설치 중...")
install_packages()
print("✅ 환경 설정 완료!")

🚀 KoBERT 요약 성능 평가 시스템 - 완전 자동화
📦 필수 패키지 설치 중...
📦 설치 중: torch...
✅ torch 설치 완료
📦 설치 중: sentence-transformers==2.2.2...
✅ torch 설치 완료
📦 설치 중: sentence-transformers==2.2.2...
⚠️ sentence-transformers==2.2.2 설치 건너뜀: Command '['c:\\ProgramData\\anaconda3\\python.exe'...
📦 설치 중: pandas...
⚠️ sentence-transformers==2.2.2 설치 건너뜀: Command '['c:\\ProgramData\\anaconda3\\python.exe'...
📦 설치 중: pandas...
✅ pandas 설치 완료
✅ 환경 설정 완료!
✅ pandas 설치 완료
✅ 환경 설정 완료!


In [6]:
# 📚 2단계: 라이브러리 임포트 및 GPU 설정
import pandas as pd
import numpy as np
import torch
import gc
import re
import codecs
from io import StringIO
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime
from tqdm import tqdm

print("📚 라이브러리 로드 완료")

# GPU/CPU 자동 감지
print("\n🔥 디바이스 설정 중...")
if torch.cuda.is_available():
    device = torch.device("cuda")
    device_name = torch.cuda.get_device_name(0)
    batch_size = 16
    print(f"✅ GPU 감지: {device_name}")

    # GPU 메모리 확인
    memory_gb = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"📊 GPU 메모리: {memory_gb:.1f}GB")

    # 배치 크기 조정
    if memory_gb >= 8:
        batch_size = 32
    elif memory_gb >= 4:
        batch_size = 16
    else:
        batch_size = 8

else:
    device = torch.device("cpu")
    device_name = "CPU"
    batch_size = 4
    print("🖥️ CPU 모드로 실행")

print(f"🎯 최종 설정: {device_name}, 배치 크기: {batch_size}")

📚 라이브러리 로드 완료

🔥 디바이스 설정 중...
🖥️ CPU 모드로 실행
🎯 최종 설정: CPU, 배치 크기: 4


In [7]:
# 🤖 3단계: SBERT 모델 로드
print("🤖 SBERT 모델 로드 중...")
print("=" * 40)

try:
    from sentence_transformers import SentenceTransformer, util

    print("✅ sentence_transformers 임포트 성공")

    # KoBERT 모델 로드
    print("🔄 KoBERT 모델 다운로드 및 로드 중... (최초 실행시 시간 소요)")
    model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")
    model = model.to(device)

    # 테스트
    print("🧪 모델 테스트 중...")
    test_sentences = ["안녕하세요", "좋은 하루 되세요"]
    test_embeddings = model.encode(test_sentences, convert_to_tensor=True)
    test_similarity = util.pytorch_cos_sim(test_embeddings[0], test_embeddings[1])

    print(f"✅ SBERT 모델 로드 성공!")
    print(f"   모델: snunlp/KR-SBERT-V40K-klueNLI-augSTS")
    print(f"   디바이스: {test_embeddings.device}")
    print(f"   테스트 유사도: {test_similarity.item():.4f}")

    USE_SBERT = True

    # 메모리 정리
    del test_embeddings, test_similarity
    if device.type == "cuda":
        torch.cuda.empty_cache()

except Exception as e:
    print(f"❌ SBERT 로드 실패: {e}")
    print("💡 TF-IDF + Jaccard + 키워드 유사도로 평가 진행")
    USE_SBERT = False
    model = None

print(f"\n🎯 SBERT 사용: {USE_SBERT}")
print("=" * 40)

🤖 SBERT 모델 로드 중...
✅ sentence_transformers 임포트 성공
🔄 KoBERT 모델 다운로드 및 로드 중... (최초 실행시 시간 소요)
✅ sentence_transformers 임포트 성공
🔄 KoBERT 모델 다운로드 및 로드 중... (최초 실행시 시간 소요)
🧪 모델 테스트 중...
✅ SBERT 모델 로드 성공!
   모델: snunlp/KR-SBERT-V40K-klueNLI-augSTS
   디바이스: cpu
   테스트 유사도: 0.3059

🎯 SBERT 사용: True
🧪 모델 테스트 중...
✅ SBERT 모델 로드 성공!
   모델: snunlp/KR-SBERT-V40K-klueNLI-augSTS
   디바이스: cpu
   테스트 유사도: 0.3059

🎯 SBERT 사용: True


In [8]:
# 📁 4단계: 데이터 로드 및 전처리
print("📁 데이터 로드 및 전처리")
print("=" * 40)


# 안전한 CSV 읽기 함수
def safe_read_csv(file_path):
    """BOM 처리가 포함된 안전한 CSV 읽기"""
    try:
        with open(file_path, "rb") as f:
            raw_data = f.read()
        if raw_data.startswith(codecs.BOM_UTF8):
            raw_data = raw_data[len(codecs.BOM_UTF8) :]
        content = raw_data.decode("utf-8", errors="replace")
        return pd.read_csv(StringIO(content))
    except:
        for encoding in ["utf-8-sig", "utf-8", "cp949", "euc-kr"]:
            try:
                return pd.read_csv(file_path, encoding=encoding)
            except:
                continue
        raise ValueError(f"CSV 파일 읽기 실패: {file_path}")


# 최적 컬럼 감지 함수
def detect_best_column(df, keywords):
    """키워드 기반 최적 컬럼 자동 감지"""
    for keyword in keywords:
        for col in df.columns:
            if keyword in col:
                return col

    # 키워드 매칭 실패시 가장 긴 텍스트 컬럼 선택
    text_cols = []
    for col in df.columns:
        if df[col].dtype == "object":
            avg_len = df[col].astype(str).str.len().mean()
            if avg_len > 50:
                text_cols.append((col, avg_len))

    if text_cols:
        return max(text_cols, key=lambda x: x[1])[0]
    else:
        return df.columns[0]


# 텍스트 전처리 함수
def preprocess_text(text):
    """텍스트 정규화 및 전처리"""
    if pd.isna(text) or text == "":
        return ""
    text = str(text).strip()
    text = re.sub(r"\s+", " ", text)  # 공백 정규화
    text = re.sub(r"[^\w\s가-힣]", "", text)  # 특수문자 제거
    return text


# 파일 로드
data_folder = "data"
original_file = os.path.join(data_folder, "crawling_origin.csv")
summary_file = os.path.join(data_folder, "crawling_origin_with_summary.csv")

print(f"📂 원본 파일: {original_file}")
print(f"📂 요약 파일: {summary_file}")

if os.path.exists(original_file) and os.path.exists(summary_file):
    # 데이터 로드
    print("\n🔄 데이터 로드 중...")
    original_df = safe_read_csv(original_file)
    summary_df = safe_read_csv(summary_file)

    print(f"📊 원본 데이터: {len(original_df):,}개 행")
    print(f"📊 요약 데이터: {len(summary_df):,}개 행")

    # 컬럼 감지
    original_col = detect_best_column(
        original_df, ["본문", "content", "text", "article"]
    )
    summary_col = detect_best_column(
        summary_df, ["요약문", "요약", "summary", "abstract"]
    )

    print(f"\n📋 감지된 컬럼:")
    print(f"   원본: '{original_col}'")
    print(f"   요약: '{summary_col}'")

    # 처리 가능한 데이터 크기
    min_len = min(len(original_df), len(summary_df))
    print(f"\n🎯 처리 가능한 데이터: {min_len:,}개 쌍")

    print("✅ 데이터 로드 완료!")

else:
    print("❌ 데이터 파일을 찾을 수 없습니다!")
    print(f"   원본: {'존재' if os.path.exists(original_file) else '없음'}")
    print(f"   요약: {'존재' if os.path.exists(summary_file) else '없음'}")
    raise FileNotFoundError("필요한 CSV 파일이 없습니다.")

print("=" * 40)

📁 데이터 로드 및 전처리
📂 원본 파일: data\crawling_origin.csv
📂 요약 파일: data\crawling_origin_with_summary.csv

🔄 데이터 로드 중...


📊 원본 데이터: 10,952개 행
📊 요약 데이터: 9,066개 행

📋 감지된 컬럼:
   원본: '본문'
   요약: '요약문'

🎯 처리 가능한 데이터: 9,066개 쌍
✅ 데이터 로드 완료!


In [9]:
# 🧮 5단계: 유사도 계산 함수 정의
print("🧮 유사도 계산 함수 정의")
print("=" * 40)


def calculate_sbert_similarity(originals, summaries, batch_size=16):
    """SBERT 유사도 계산 (GPU/CPU 자동 최적화)"""
    if not USE_SBERT or model is None:
        print("⚠️ SBERT 모델이 없어 0으로 대체")
        return np.zeros(len(originals))

    print(f"🚀 SBERT 유사도 계산 시작 ({device})")
    similarities = []
    total_batches = (len(originals) + batch_size - 1) // batch_size

    with torch.no_grad():
        for i in tqdm(
            range(0, len(originals), batch_size), desc=f"SBERT {device.type.upper()}"
        ):
            batch_orig = originals[i : i + batch_size]
            batch_summ = summaries[i : i + batch_size]

            try:
                orig_embeddings = model.encode(
                    batch_orig, convert_to_tensor=True, device=device
                )
                summ_embeddings = model.encode(
                    batch_summ, convert_to_tensor=True, device=device
                )

                batch_similarities = util.pytorch_cos_sim(
                    orig_embeddings, summ_embeddings
                ).diag()
                similarities.extend(batch_similarities.cpu().numpy())

                # 메모리 정리
                del orig_embeddings, summ_embeddings, batch_similarities
                if device.type == "cuda":
                    torch.cuda.empty_cache()

            except Exception as e:
                print(f"❌ 배치 {i//batch_size + 1} 실패: {e}")
                similarities.extend([0.0] * len(batch_orig))

    return np.array(similarities)


def calculate_tfidf_similarity(originals, summaries):
    """TF-IDF 유사도 계산"""
    print("🔄 TF-IDF 유사도 계산 중...")
    try:
        vectorizer = TfidfVectorizer(max_features=1000, ngram_range=(1, 2))
        all_texts = originals + summaries
        tfidf_matrix = vectorizer.fit_transform(all_texts)

        orig_matrix = tfidf_matrix[: len(originals)]
        summ_matrix = tfidf_matrix[len(originals) :]

        similarities = []
        for i in tqdm(range(len(originals)), desc="TF-IDF"):
            sim = cosine_similarity(orig_matrix[i], summ_matrix[i])[0][0]
            similarities.append(sim)

        return np.array(similarities)
    except Exception as e:
        print(f"❌ TF-IDF 실패: {e}")
        return np.zeros(len(originals))


def calculate_jaccard_similarity(originals, summaries):
    """Jaccard 유사도 계산"""
    print("🔄 Jaccard 유사도 계산 중...")
    similarities = []
    for orig, summ in tqdm(
        zip(originals, summaries), total=len(originals), desc="Jaccard"
    ):
        orig_words = set(orig.split())
        summ_words = set(summ.split())

        if len(orig_words) == 0 and len(summ_words) == 0:
            similarities.append(1.0)
        elif len(orig_words) == 0 or len(summ_words) == 0:
            similarities.append(0.0)
        else:
            intersection = len(orig_words & summ_words)
            union = len(orig_words | summ_words)
            similarities.append(intersection / union)

    return np.array(similarities)


def calculate_keyword_similarity(originals, summaries, top_k=10):
    """키워드 기반 유사도 계산"""
    print("🔄 키워드 유사도 계산 중...")
    similarities = []
    for orig, summ in tqdm(
        zip(originals, summaries), total=len(originals), desc="키워드"
    ):
        orig_words = Counter(orig.split())
        summ_words = Counter(summ.split())

        orig_top = set([word for word, _ in orig_words.most_common(top_k)])
        summ_top = set([word for word, _ in summ_words.most_common(top_k)])

        if len(orig_top) == 0 and len(summ_top) == 0:
            similarities.append(1.0)
        elif len(orig_top) == 0 or len(summ_top) == 0:
            similarities.append(0.0)
        else:
            intersection = len(orig_top & summ_top)
            union = len(orig_top | summ_top)
            similarities.append(intersection / union)

    return np.array(similarities)


print("✅ 유사도 계산 함수 준비 완료!")
print("=" * 40)

🧮 유사도 계산 함수 정의
✅ 유사도 계산 함수 준비 완료!


In [10]:
# 🚀 6단계: 전체 데이터 평가 실행
print("🚀 전체 데이터 평가 실행")
print("=" * 60)

# 데이터 전처리
print("🔄 데이터 전처리 중...")
originals = []
summaries = []
valid_count = 0
skipped_count = 0

for i in tqdm(range(min_len), desc="전처리"):
    try:
        orig_text = preprocess_text(original_df.iloc[i][original_col])
        summ_text = preprocess_text(summary_df.iloc[i][summary_col])

        if len(orig_text) >= 10 and len(summ_text) >= 5:
            originals.append(orig_text)
            summaries.append(summ_text)
            valid_count += 1
        else:
            skipped_count += 1
    except:
        skipped_count += 1

print(f"\n📊 전처리 결과:")
print(f"   ✅ 유효 데이터: {valid_count:,}개")
print(f"   ⚠️ 제외 데이터: {skipped_count:,}개")
print(f"   📈 유효율: {valid_count/(valid_count+skipped_count)*100:.1f}%")

if valid_count == 0:
    raise ValueError("유효한 데이터가 없습니다!")

# 유사도 계산
print(f"\n🧮 {valid_count:,}개 데이터 유사도 계산 시작...")
print("=" * 50)

# 1. SBERT 유사도
sbert_scores = calculate_sbert_similarity(originals, summaries, batch_size)
print(f"✅ SBERT 평균: {np.mean(sbert_scores):.4f}")

# 2. TF-IDF 유사도
tfidf_scores = calculate_tfidf_similarity(originals, summaries)
print(f"✅ TF-IDF 평균: {np.mean(tfidf_scores):.4f}")

# 3. Jaccard 유사도
jaccard_scores = calculate_jaccard_similarity(originals, summaries)
print(f"✅ Jaccard 평균: {np.mean(jaccard_scores):.4f}")

# 4. 키워드 유사도
keyword_scores = calculate_keyword_similarity(originals, summaries)
print(f"✅ 키워드 평균: {np.mean(keyword_scores):.4f}")

print("\n" + "=" * 50)
print("🎉 모든 유사도 계산 완료!")

🚀 전체 데이터 평가 실행
🔄 데이터 전처리 중...


전처리: 100%|██████████| 9066/9066 [00:01<00:00, 8001.59it/s]



📊 전처리 결과:
   ✅ 유효 데이터: 9,022개
   ⚠️ 제외 데이터: 44개
   📈 유효율: 99.5%

🧮 9,022개 데이터 유사도 계산 시작...
🚀 SBERT 유사도 계산 시작 (cpu)


SBERT CPU: 100%|██████████| 2256/2256 [14:26<00:00,  2.60it/s]



🚀 전체 데이터 평가 실행
🔄 데이터 전처리 중...


...

✅ TF-IDF 평균: 0.0271
🔄 Jaccard 유사도 계산 중...


Jaccard: 100%|██████████| 9022/9022 [00:00<00:00, 91510.83it/s]


✅ Jaccard 평균: 0.0129
🔄 키워드 유사도 계산 중...


키워드: 100%|██████████| 9022/9022 [00:00<00:00, 35190.07it/s]

✅ 키워드 평균: 0.0152

🎉 모든 유사도 계산 완료!


In [11]:
# 📊 7단계: 결과 분석 및 종합 점수 계산
print("📊 결과 분석 및 종합 점수 계산")
print("=" * 50)

# 변수 존재 확인 및 기본값 설정
if "sbert_scores" not in globals():
    print("⚠️ sbert_scores가 정의되지 않음 - 6단계를 먼저 실행해주세요")
    print("🔄 기본 더미 데이터로 진행...")
    sbert_scores = np.array([0.0] * 100)  # 더미 데이터
    tfidf_scores = np.array([0.0] * 100)
    jaccard_scores = np.array([0.0] * 100)
    keyword_scores = np.array([0.0] * 100)
    originals = ["더미 원본 텍스트"] * 100
    summaries = ["더미 요약 텍스트"] * 100
    valid_count = 100

# 가중치 설정
weights = {
    "sbert": 0.4,  # KoBERT가 가장 중요
    "tfidf": 0.3,  # TF-IDF도 중요
    "jaccard": 0.2,  # Jaccard는 보조
    "keyword": 0.1,  # 키워드는 참고용
}

# 종합 점수 계산
composite_scores = (
    weights["sbert"] * sbert_scores
    + weights["tfidf"] * tfidf_scores
    + weights["jaccard"] * jaccard_scores
    + weights["keyword"] * keyword_scores
)

# 통계 계산
results = {
    "SBERT": sbert_scores,
    "TF-IDF": tfidf_scores,
    "Jaccard": jaccard_scores,
    "Keyword": keyword_scores,
    "Composite": composite_scores,
}

print("📈 상세 통계:")
print("-" * 70)
print(
    f"{'방법':<12} {'평균':<8} {'표준편차':<8} {'최소값':<8} {'최대값':<8} {'중간값':<8}"
)
print("-" * 70)

for method, scores in results.items():
    mean_score = np.mean(scores)
    std_score = np.std(scores)
    min_score = np.min(scores)
    max_score = np.max(scores)
    median_score = np.median(scores)

    print(
        f"{method:<12} {mean_score:<8.4f} {std_score:<8.4f} {min_score:<8.4f} {max_score:<8.4f} {median_score:<8.4f}"
    )

# 성능 등급 분석
best_scores = composite_scores
excellent = np.sum(best_scores >= 0.80)
good = np.sum((best_scores >= 0.60) & (best_scores < 0.80))
average = np.sum((best_scores >= 0.40) & (best_scores < 0.60))
poor = np.sum(best_scores < 0.40)

print(f"\n🏆 성능 등급 분포 (총 {len(best_scores):,}개):")
print(f"   🥇 우수 (≥0.80): {excellent:,}개 ({excellent/len(best_scores)*100:5.1f}%)")
print(f"   🥈 양호 (0.60-0.80): {good:,}개 ({good/len(best_scores)*100:5.1f}%)")
print(f"   🥉 보통 (0.40-0.60): {average:,}개 ({average/len(best_scores)*100:5.1f}%)")
print(f"   📉 미흡 (<0.40): {poor:,}개 ({poor/len(best_scores)*100:5.1f}%)")

# 최고/최저 성능 샘플
best_idx = np.argmax(best_scores)
worst_idx = np.argmin(best_scores)

print(f"\n🎯 성능 샘플:")
print(f"   🏆 최고 점수: {best_scores[best_idx]:.4f}")
print(f"      원본: {originals[best_idx][:100]}...")
print(f"      요약: {summaries[best_idx][:100]}...")

print(f"\n   📉 최저 점수: {best_scores[worst_idx]:.4f}")
print(f"      원본: {originals[worst_idx][:100]}...")
print(f"      요약: {summaries[worst_idx][:100]}...")

# 전체 평가 결과
avg_score = np.mean(best_scores)
print(f"\n🎯 전체 평가 결과:")
print(f"   📊 종합 평균 점수: {avg_score:.4f}")

if avg_score >= 0.70:
    grade = "A (우수)"
    comment = "매우 높은 품질의 요약 성능을 보입니다!"
elif avg_score >= 0.60:
    grade = "B (양호)"
    comment = "양호한 요약 성능입니다. 추가 개선 여지가 있습니다."
elif avg_score >= 0.50:
    grade = "C (보통)"
    comment = "평균적인 성능입니다. 개선이 필요합니다."
else:
    grade = "D (미흡)"
    comment = "성능 개선이 시급합니다."

print(f"   🏆 성능 등급: {grade}")
print(f"   💡 평가: {comment}")

print("\n" + "=" * 50)
print("✅ 결과 분석 완료!")

📊 결과 분석 및 종합 점수 계산
📈 상세 통계:
----------------------------------------------------------------------
방법           평균       표준편차     최소값      최대값      중간값     
----------------------------------------------------------------------
SBERT        0.3085   0.1164   -0.0532  0.8945   0.3029  
TF-IDF       0.0271   0.0618   0.0000   0.8519   0.0000  
Jaccard      0.0129   0.0267   0.0000   0.6538   0.0000  
Keyword      0.0152   0.0380   0.0000   1.0000   0.0000  
Composite    0.1356   0.0617   -0.0213  0.7733   0.1290  

🏆 성능 등급 분포 (총 9,022개):
   🥇 우수 (≥0.80): 0개 (  0.0%)
   🥈 양호 (0.60-0.80): 10개 (  0.1%)
   🥉 보통 (0.40-0.60): 21개 (  0.2%)
   📉 미흡 (<0.40): 8,991개 ( 99.7%)

🎯 성능 샘플:
   🏆 최고 점수: 0.7733
      원본: 이재명 대통령은 국회에 강선우 여성가족부 장관 후보자를 포함해 국방부와 국가보훈부 통일부 장관 후보자에 대한 인사청문보고서를 내일까지 재송부해 달라고 요청했습니다 강유정 대통령실 ...
      요약: 이재명 이재명 대통령은 국회에 강선우 여성가족부 장관 후보자를 포함해 국방부와 국가보훈부 통일부 장관 후보자에 대한 인사청문보고서를 내일까지 재송부해 달라고 요청했습니다...

   📉 최저 점수: -0.0213
      원본:  앵커  이재명 대통령이 주로 장관급 국무위원들이 참석해왔던 국무회의 관행에 큰 변

In [12]:
# 💾 8단계: 결과 저장
print("💾 평가 결과 저장")
print("=" * 40)

# 결과 데이터프레임 생성
results_data = {
    "original_text": [
        text[:200] + "..." if len(text) > 200 else text for text in originals
    ],
    "summary_text": [
        text[:200] + "..." if len(text) > 200 else text for text in summaries
    ],
    "sbert_score": sbert_scores,
    "tfidf_score": tfidf_scores,
    "jaccard_score": jaccard_scores,
    "keyword_score": keyword_scores,
    "composite_score": composite_scores,
    "original_length": [len(text) for text in originals],
    "summary_length": [len(text) for text in summaries],
    "compression_ratio": [len(s) / len(o) * 100 for o, s in zip(originals, summaries)],
}

results_df = pd.DataFrame(results_data)

# 파일명 생성
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"kobert_complete_results_{timestamp}.csv"
summary_file = f"kobert_complete_summary_{timestamp}.txt"

# CSV 저장
results_df.to_csv(output_file, index=False, encoding="utf-8-sig")
print(f"✅ 상세 결과 저장: {output_file}")

# 요약 통계 저장
with open(summary_file, "w", encoding="utf-8") as f:
    f.write(f"KoBERT 요약 성능 평가 결과 (완전 자동화)\n")
    f.write(f"=" * 50 + "\n")
    f.write(f"평가 일시: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"평가 데이터: {valid_count:,}개\n")
    f.write(f"디바이스: {device_name}\n")
    f.write(f"SBERT 사용: {USE_SBERT}\n")
    f.write(f"\n성능 통계:\n")
    f.write(f"  SBERT 평균: {np.mean(sbert_scores):.4f}\n")
    f.write(f"  TF-IDF 평균: {np.mean(tfidf_scores):.4f}\n")
    f.write(f"  Jaccard 평균: {np.mean(jaccard_scores):.4f}\n")
    f.write(f"  키워드 평균: {np.mean(keyword_scores):.4f}\n")
    f.write(f"  종합 평균: {avg_score:.4f}\n")
    f.write(f"\n성능 등급: {grade}\n")
    f.write(f"평가: {comment}\n")
    f.write(f"\n등급 분포:\n")
    f.write(
        f"  우수 (≥0.80): {excellent:,}개 ({excellent/len(best_scores)*100:5.1f}%)\n"
    )
    f.write(f"  양호 (0.60-0.80): {good:,}개 ({good/len(best_scores)*100:5.1f}%)\n")
    f.write(
        f"  보통 (0.40-0.60): {average:,}개 ({average/len(best_scores)*100:5.1f}%)\n"
    )
    f.write(f"  미흡 (<0.40): {poor:,}개 ({poor/len(best_scores)*100:5.1f}%)\n")

print(f"✅ 요약 통계 저장: {summary_file}")

# 메모리 정리
if device.type == "cuda":
    torch.cuda.empty_cache()
gc.collect()

print(f"\n🎉 전체 평가 프로세스 완료!")
print(f"📁 결과 파일: {output_file}")
print(f"📋 요약 파일: {summary_file}")
print(f"🏆 최종 점수: {avg_score:.4f} ({grade})")
print(f"📊 평가 데이터: {valid_count:,}개")
print(f"🎯 SBERT 사용: {'✅' if USE_SBERT else '❌'}")
print(f"🖥️ 디바이스: {device_name}")
print("=" * 60)
print("🎊 모든 과정이 성공적으로 완료되었습니다!")

💾 평가 결과 저장
✅ 상세 결과 저장: kobert_complete_results_20250805_134036.csv
✅ 요약 통계 저장: kobert_complete_summary_20250805_134036.txt

🎉 전체 평가 프로세스 완료!
📁 결과 파일: kobert_complete_results_20250805_134036.csv
📋 요약 파일: kobert_complete_summary_20250805_134036.txt
🏆 최종 점수: 0.1356 (D (미흡))
📊 평가 데이터: 9,022개
🎯 SBERT 사용: ✅
🖥️ 디바이스: CPU
🎊 모든 과정이 성공적으로 완료되었습니다!

🎉 전체 평가 프로세스 완료!
📁 결과 파일: kobert_complete_results_20250805_134036.csv
📋 요약 파일: kobert_complete_summary_20250805_134036.txt
🏆 최종 점수: 0.1356 (D (미흡))
📊 평가 데이터: 9,022개
🎯 SBERT 사용: ✅
🖥️ 디바이스: CPU
🎊 모든 과정이 성공적으로 완료되었습니다!
